In [162]:
# load data and write out sentence and target
import pandas as pd

loaded_set = pd.read_excel("Dataset/"+"training.xlsx")
loaded_set['Sentence'][0]

'Weitergehende Sicherungsmaßnahmen können eine Videoüberwachung und eine Zugangskontrolle durch einen Türöffner sein, denn viele GAA befinden sich in Vorräumen der Geschäftsstellen der Banken, sodass sie auch außerhalb der Schalteröffnungszeiten zugänglich sind.'

In [161]:
tokens_num=[]
for sen in loaded_set['Sentence']:
    tokenized = (tokenizer.tokenize(sen)) 
    tokens_num.append( ['[CLS]'] + tokenized + ['[SEP]']) 
    
# get max_seq length    
lens = [len(i) for i in tokens_num]
max_seq_length = max(lens)
max_seq_length = int(1.5*max_seq_length)
tokens_num[0]

['[CLS]',
 'Weiter',
 '##gehende',
 'Sicherungs',
 '##maßnahmen',
 'können',
 'eine',
 'Video',
 '##überwachung',
 'und',
 'eine',
 'Zugangs',
 '##kontrolle',
 'durch',
 'einen',
 'Tür',
 '##öff',
 '##ner',
 'sein',
 ',',
 'denn',
 'viele',
 'G',
 '##AA',
 'befinden',
 'sich',
 'in',
 'Vor',
 '##räumen',
 'der',
 'Geschäfts',
 '##stellen',
 'der',
 'Banken',
 ',',
 'sodass',
 'sie',
 'auch',
 'außerhalb',
 'der',
 'Schalter',
 '##öffnungs',
 '##zeiten',
 'zugänglich',
 'sind',
 '.',
 '[SEP]']

In [163]:
tokenizer.convert_tokens_to_ids(tokens_num[0])

[102,
 1784,
 13183,
 28847,
 4686,
 618,
 261,
 4770,
 20815,
 136,
 261,
 21093,
 11600,
 387,
 397,
 2451,
 706,
 432,
 290,
 818,
 1398,
 1358,
 159,
 10695,
 3857,
 251,
 153,
 445,
 7721,
 125,
 2484,
 984,
 125,
 8232,
 818,
 7415,
 307,
 313,
 5729,
 125,
 28802,
 17893,
 4083,
 10370,
 341,
 566,
 103]

In [109]:
import tensorflow_hub as hub
from transformers import AutoModel, AutoTokenizer
# german tokens for bert
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
#model = AutoModel.from_pretrained("dbmdz/bert-base-german-cased")

In [110]:
import numpy as np
def encode_names(n, tokenizer):
   tokens = list(tokenizer.tokenize(n))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(string_list, tokenizer, max_seq_length):
  num_examples = len(string_list)
  
  string_tokens = tf.ragged.constant([
      encode_names(n, tokenizer) for n in np.array(string_list)])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*string_tokens.shape[0]
  input_word_ids = tf.concat([cls, string_tokens], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor(shape=(None, max_seq_length))

  type_cls = tf.zeros_like(cls)
  type_tokens = tf.ones_like(string_tokens)
  input_type_ids = tf.concat(
      [type_cls, type_tokens], axis=-1).to_tensor(shape=(None, max_seq_length))

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(shape=(None, max_seq_length)),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs

In [121]:
x = loaded_set['Sentence']
y = loaded_set['MOS']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=32)
y_train = round(y_train, 2)
y_test = round(y_test, 2)


49     2.30
863    2.11
607    4.07
853    2.62
74     3.29
       ... 
88     1.89
892    1.31
310    3.20
555    2.11
727    4.00
Name: MOS, Length: 720, dtype: float64

In [171]:

X_train = bert_encode(x_train, tokenizer, max_seq_length)
X_test = bert_encode(x_test, tokenizer, max_seq_length)
X_train['input_word_ids'][80]

<tf.Tensor: shape=(144,), dtype=int32, numpy=
array([  102,  2135,   153,   128,  5500,   105,   744,   136,  3994,
         285,   128, 13517,  6026,   125,  9348,   809,   313,   153,
         125, 23568,  4030,  3627,   809,   842,   222, 23306,  2178,
        1579,  1084,   566,   103,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,   

In [178]:
import tensorflow as tf
num_class = 1  # Based on available class selection
max_seq_length = max_seq_length  # we calculated this a couple cells ago

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")

bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

#output_drop = tf.keras.layers.Dropout(rate=0.1)(pooled_output)
output_dense_1 = tf.keras.layers.Dense(256, activation='relu', name='dense_1')(pooled_output)
output_dense_2 = tf.keras.layers.Dense(128, activation='relu', name='dense_2')(output_dense_1)
output_dense_3 = tf.keras.layers.Dense(128, activation='relu', name='dense_3')(output_dense_2)
output = tf.keras.layers.Dense(1, activation='softmax', name='output')(output_dense_3)

model = tf.keras.Model(
    inputs={
        'input_word_ids':input_word_ids,
        'input_mask':input_mask,
        'input_type_ids':segment_ids
    },
    outputs = output
)

In [179]:


epochs = 1
batch_size = 8  # select based on your GPU resources


train_data_size = len(y_train)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)




In [180]:
model.compile(optimizer=optimizer,
            loss='mse'
            )

In [181]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 144)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 144)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 144)]        0                                            
__________________________________________________________________________________________________
keras_layer_10 (KerasLayer)     [(None, 768), (None, 177853441   input_word_ids[0][0]             
                                                                 input_mask[0][0]          

In [189]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1)

Epoch 1/5
90/90 [==============================] - 709s 8s/step - loss: 5.5403 - val_loss: 5.0111
Epoch 2/5
90/90 [==============================] - 653s 7s/step - loss: 5.5403 - val_loss: 5.0111
Epoch 3/5
90/90 [==============================] - 636s 7s/step - loss: 5.5403 - val_loss: 5.0111
Epoch 4/5
90/90 [==============================] - 640s 7s/step - loss: 5.5404 - val_loss: 5.0111
Epoch 5/5
90/90 [==============================] - 638s 7s/step - loss: 5.5403 - val_loss: 5.0111


In [188]:
pred = model.predict(X_test)
rounded_pred = np.around(pred, decimals=2)
rounded_pred

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [187]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse(rounded_pred, y_test.values)

2.238544045678897